In [ ]:
dir_root = '/user/centos/'
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
from pyspark.sql import functions as sf

### *Exercises*

1. Load the Heroes of the Storm dataset with read csv ('data/heroes.csv' in the hdfs homedir of user centos)
2. check the dtypes: 
    - what do you notice?
    - Fix it.
2. Manually explore the data and look for corrupted/malformed data
3. Find a way to remove these corrupted rows
3. Which hero has the most hp?
4. Add a column with the 'attack_momentum', computed as attack * attack_spd
5. Which role on average has the highest attack?
6. Figure out which roles and attack_type frequently co-occur
7. Deliver a dataframe with the highest attack per role 
8. export to Pandas

Bonus
9. make a function that accepts a dataframe and a list colnames. Let it return the mean and stddev of the columns 
10. apply the function to the hp and attack column such that the result has columns:

`hp_mean, hp_stddev, attack_mean, attack_stddev`

In [ ]:
# 1. Load
ddf_heroes = spark.read.csv(dir_root + 'data/heroes.csv', header = True)

In [ ]:
# 2a. Types
ddf_heroes.dtypes

In [ ]:
# 2b. 
ddf_heroes = ddf_heroes.select( sf.col("_c0"), sf.col("name"), sf.col("hp").cast('int').alias("hp"), sf.col("attack").cast('int').alias("attack"), sf.col("attack_spd").cast('float').alias("attack_spd"), sf.col("attack_type"), sf.col("role") )

In [ ]:
# 3.
ddf_heroes.show(50)

In [ ]:
# 4. 
ddf_heroes = ddf_heroes.filter(sf.col('name') != 'Gall') # remove Gall

In [ ]:
# 5.
ddf_heroes.sort(sf.desc('hp')).show(5)

In [ ]:
# 6. 
ddf_heroes = ddf_heroes.withColumn('attack_momentum', sf.col('attack') * sf.col('attack_spd'))
ddf_heroes.show(5)

In [ ]:
# 7.
ddf_heroes.groupBy('role').agg(sf.mean('attack')).sort(sf.desc('avg(attack)')).show()

In [ ]:
# 8. 
ddf_heroes.groupBy('role', 'attack_type').count().show()

In [ ]:
# 9.

ddf_attack = ddf_heroes.groupBy('role').agg({'attack': 'max'})
ddf_attack.show()

In [ ]:
# 10.
df = ddf_attack.toPandas()
df

In [ ]:
## Add Bonus questions